In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Brain_Tumor_Dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.epochs,
            params_batch_size=params.batch_size,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.input_shape
        )

        return training_config

In [10]:
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        # Use Pathlib's `/` operator instead of os.path.join
        tb_running_log_dir = self.config.tensorboard_root_log_dir / f"tb_logs_at_{timestamp}"

        # Convert Pathlib Path to string for TensorBoard callback
        return tf.keras.callbacks.TensorBoard(log_dir=str(tb_running_log_dir))
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

#### 1.Class Definition and Constructor:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

What's happening:

When we create an object (an instance) of the Training class, it uses the config to get all the necessary settings.
The config object holds important details like where the model is saved, where the data is stored, how many images to process in a batch, and whether or not to apply data augmentation (random image transformations).


Let's go through the code in simple terms, with detailed explanations and real-world analogies to help you better understand what's happening in each part of the code.

Purpose of the Code:
The Training class is designed to:

Load a model that has already been trained or fine-tuned.
Prepare the data (images) for training and validation.
Train the model on the data.
Save the best version of the trained model for later use.


#### 2.Loading the Pre-Trained Model:

def get_base_model(self):
    self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

What's happening:

The method loads a pre-trained model from the file specified in the config.
This is useful because we might have already fine-tuned the model earlier, and now we are loading it to continue training or improve it further.



#### 3.Preparing the Data Generators (Images for Training and Validation):

def train_valid_generator(self):

What's happening:

This method is responsible for preparing the data (usually images) that will be fed into the model during training and validation.
It uses Keras' ImageDataGenerator to load images from a folder and optionally apply transformations to them (e.g., flipping, rotating, zooming).

It creates two generators:

Training generator: For feeding training images.

Validation generator: For feeding validation images to check the model’s performance.

=> **Data Preparation Settings:**

datagenerator_kwargs = dict()

What's happening:

Rescaling: Pixel values in images typically range from 0 to 255. Dividing by 255 brings these values between 0 and 1, which helps the model train better.

Validation Split: We split the data into training data (80%) and validation data (20%). The validation data is used to check how well the model is doing on unseen data.

=> **Data Flow Settings (How Images Are Handled):**

dataflow_kwargs = dict()

What's happening:

Target Size: All images are resized to a fixed size (e.g., 224x224 pixels) so that they can fit into the model.

Batch Size: Instead of feeding the model all images at once, we feed it a small batch of images (e.g., 32 images at a time).

Interpolation: If the image size doesn’t match exactly, it is resized using a technique called bilinear interpolation, which smooths the resizing.

=> **Validation Data Generator:**

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
self.valid_generator = valid_datagenerator.flow_from_directory()

What's happening:

Validation Data Generator: This prepares the validation data from the directory where the images are stored. It applies the rescaling and validation split defined earlier.

subset="validation": We’re telling it to use only the validation set.

shuffle=False: The validation data is not shuffled because it’s used to check performance at the end of each training epoch.

=> **Training Data Generator with Data Augmentation:**

if self.config.params_is_augmentation:
    train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator()

What's happening:

If data augmentation is enabled, we apply random transformations to the training images (e.g., rotating, flipping, zooming). This helps the model generalize better by exposing it to different variations of the same images.


=> **Training Data Generator without Augmentation:**
else:
    train_datagenerator = valid_datagenerator

What's happening:

If augmentation is not enabled, we reuse the same data generator as the validation set for training (without applying transformations).

#### 4.Saving the Model:

@staticmethod
def save_model(path: Path, model: tf.keras.Model):
    model.save(path)

What's happening:

This method saves the trained model to a specified path so that it can be reloaded later for evaluation or use in production.

#### 5.Training the Model:

def train(self, callback_list: list):
    
What's happening:

Steps per Epoch: This calculates how many batches of images the model will process in each epoch (training cycle). It’s calculated by dividing the number of samples by the batch size.

Validation Steps: Similar to steps_per_epoch, but for validation data.


=> **Training the Model with Callbacks:**

self.model.fit()

What's happening:

self.model.fit(): This is where the actual training happens. The model is trained on batches of training data, and after each epoch, its performance is evaluated on the validation data.

Callbacks: These are functions that help track progress (e.g., saving the best model or logging metrics).

=> **Saving the Trained Model:**

self.save_model()

What's happening:

After training is complete, this method saves the best version of the trained model.

In [22]:
class Training:
    def __init__(self, config: TrainingConfig):
        # Constructor: Initialize the class with the provided configuration
        # 'config' contains all the necessary settings like paths, batch sizes, image sizes, etc.
        self.config = config


    def get_base_model(self):
        # Load the pre-trained or fine-tuned model from the specified path in the config
        # The model was previously trained and saved at this path, so we are loading it here
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path  # Path to the saved model
        )

    def train_valid_generator(self):
        # This method sets up the data generators for both training and validation sets

        # Common data preprocessing settings for both training and validation datasets
        # rescale=1./255: Normalize image pixel values to a range between 0 and 1
        # validation_split=0.20: Split 20% of the data for validation, and the remaining 80% for training
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        # Settings for how the data should be processed and fed into the model
        # target_size: Resize all images to a fixed size (e.g., 224x224)
        # batch_size: Number of images processed in one batch
        # interpolation="bilinear": How images should be resized if they don't match the target size
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Excludes the channel dimension (e.g., [224, 224])
            batch_size=self.config.params_batch_size,        # Number of images per batch
            interpolation="bilinear"                        # Method for resizing images
        )

        # Creating the ImageDataGenerator for validation data (rescaling and splitting handled here)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Creating the validation generator using the flow_from_directory method
        # This generator loads the images from the specified directory and applies validation settings
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,  # Directory where the image data is stored
            subset="validation", 
            color_mode="grayscale",                 # Use the validation split (20% of data)
            shuffle=False,                        # No need to shuffle validation data
            **dataflow_kwargs                     # Apply target size and batch size settings
        )

        # If data augmentation is enabled, create a training data generator with augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,               # Rotate images randomly up to 40 degrees
                horizontal_flip=True,            # Flip images horizontally
                width_shift_range=0.2,           # Shift images horizontally by up to 20%
                height_shift_range=0.2,          # Shift images vertically by up to 20%
                shear_range=0.2,                 # Apply shear transformations to the images
                zoom_range=0.2,                  # Zoom in or out randomly up to 20%
                **datagenerator_kwargs           # Apply the common data preprocessing settings (e.g., rescaling)
            )
        else:
            # If augmentation is not enabled, use the same generator as the validation generator
            train_datagenerator = valid_datagenerator

        # Creating the training generator to load images in batches and shuffle them for better training
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,  # Directory where the image data is stored
            subset="training",
            color_mode="grayscale",                   # Use the training split (80% of data)
            shuffle=True,                        # Shuffle training data to ensure randomness
            **dataflow_kwargs                    # Apply target size and batch size settings
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the trained model to the specified path
        # This function ensures that the model's architecture, weights, and training configuration are saved
        model.save(path)

    def train(self, callback_list: list):
        # Training method to fit the model on the training data and validate it on the validation data

        # Calculate how many steps (batches) the model needs to go through for each epoch during training
        # steps_per_epoch: Total number of samples divided by the batch size (training data)
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        
        # Similarly, calculate the number of validation steps per epoch
        # validation_steps: Total number of validation samples divided by the batch size (validation data)
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Train the model using the .fit() method
        # - train_generator: Feeds training data in batches to the model
        # - epochs: Number of complete passes through the training dataset
        # - steps_per_epoch: Number of training steps in each epoch
        # - validation_steps: Number of validation steps after each epoch
        # - validation_data: Feeds validation data to evaluate the model’s performance
        # - callbacks: A list of callbacks that will be applied during training (e.g., saving the best model, TensorBoard logging)
        self.model.fit(
            self.train_generator,               # Training data generator
            epochs=self.config.params_epochs,    # Number of epochs (how many times the model sees the entire dataset)
            steps_per_epoch=self.steps_per_epoch,  # Number of training steps per epoch
            validation_steps=self.validation_steps, # Number of validation steps per epoch
            validation_data=self.valid_generator,   # Validation data generator
            callbacks=callback_list                 # List of callbacks (e.g., saving the model, logging metrics)
        )

        # Save the trained model after the training process is complete
        self.save_model(
            path=self.config.trained_model_path,  # Save path for the final model
            model=self.model                      # The trained model
        )


In [24]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-09-13 13:44:22,317: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-13 13:44:22,328: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-13 13:44:22,332: INFO: common: created directory at: artifacts]
[2024-09-13 13:44:22,335: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-13 13:44:22,338: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-09-13 13:44:22,342: INFO: common: created directory at: artifacts\training]
Found 117 images belonging to 2 classes.
Found 468 images belonging to 2 classes.
Epoch 1/10
14/14 [==============================] - 208s 14s/step - loss: 0.6975 - accuracy: 0.5413 - val_loss: 0.6875 - val_accuracy: 0.6250
Epoch 2/10
14/14 [==============================] - 358s 27s/step - loss: 0.6602 - accuracy: 0.7179 - val_loss: 0.6788 - val_accuracy: 0.6250
Epoch 3/10
 5/14 [=========>....................] - ETA: 1:43 - loss: 0.6266 - accuracy

KeyboardInterrupt: 